In [ ]:
"""
This code evaluates the accuracy of the base ResNet18 model on different classes of pulses and then only trains fc blocks on these classes.
The FC blocks corrsponding to different classes are then concatenated together and saved in a dataframe 
"""
import itertools
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import random
import gzip, struct, time, itertools
from torchvision import datasets
import argparse
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import itertools
import sys
import time
%matplotlib inline
for p in range(0,10):
    def load_pretrained(dict_path):
        """
        Load a ResNet-18 model from `torchvision.models` without pre-trained weights. 
        """
        Resnet18 = models.resnet18(pretrained=False)
        if (dict_path != None):
            Resnet18.load_state_dict(torch.load(dict_path))
        for param in Resnet18.parameters():
            param.requires_grad = True
        n_inputs = Resnet18.fc.in_features
        Resnet18.fc = torch.nn.Linear(512,1000)
        return Resnet18

    def clear_checkpoint(checkpoint_dir):
        """
        Delete all checkpoints in directory.
        """
        filelist = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pth.tar")]
        for f in filelist:
            os.remove(os.path.join(checkpoint_dir, f))

        print("Checkpoint successfully removed")

    class FROGDataset:
        """
        Dog Dataset.
        """
        def __init__(self, batch_size=4):
            self.batch_size = batch_size
            self.train_dataset,self.train_label,self.val_dataset,self.val_label = self.get_train_numpy()
            self.train_loader,self.val_loader = self.get_dataloaders()

        def get_train_numpy(self):
            df = pd.read_pickle('/content/drive/MyDrive/Test_domain_generalization/Pyret_Labels_for_CNN_duration_2000_bin1_%02d.pkl' % (p + 5))

            self.train_dataset = np.tile(np.expand_dims(df.iloc[0,1][:400,:,:],axis = 1),[1,3,1,1])
            self.train_label = df.iloc[0,0][:400,:]
            self.val_dataset = np.tile(np.expand_dims(df.iloc[0,1][:400,:,:],axis = 1),[1,3,1,1])
            self.val_label = df.iloc[0,0][:400,:]
            return self.train_dataset,self.train_label,self.val_dataset,self.val_label
         
        def get_dataloaders(self):
            # train set
            self.train_dataset = self.train_dataset.astype(np.double)
            train_label = self.train_label 
            self.train_label  = self.train_label.astype(np.double)
            train_set = torch.tensor(self.train_dataset)
            train_label = torch.tensor(self.train_label)
            dataset = torch.utils.data.TensorDataset(train_set, train_label)
            train_loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
            print(type(train_loader))
            # validation set
            self.val_dataset  = self.val_dataset.astype(np.double)
            self.val_label = self.val_label.astype(np.double)
            val_set = torch.tensor(self.val_dataset)
            val_label = torch.tensor(self.val_label)
            datasetval = torch.utils.data.TensorDataset(val_set, val_label)
            val_loader = torch.utils.data.DataLoader(datasetval, batch_size=self.batch_size, shuffle=False)
            return train_loader,val_loader

    dataset = FROGDataset()

    def evaluate_loop(data_loader, model, criterion):
        model.eval() 
        y_true, y_pred, running_loss = [], [], []
        n=0
        for X, y in data_loader:  
            with torch.no_grad():
                output = model(X)  # output is torch tensor
                y_true.append(y)
                y_pred.append(output)
                n += 1
                if criterion is not None:
                    running_loss.append(criterion(output, y).item() * X.size(0))  # The output dimension: [batchsize,num of classes]
        model.train() 
        y_true, y_pred = torch.cat(y_true), torch.cat(y_pred) # convert y_true from list to torch tensor
        return y_true, y_pred, running_loss,n,output,y

    def restore_checkpoint(model, checkpoint_dir, cuda=False, force=False, pretrain=False):
        """
        If a checkpoint exists, restores the PyTorch model from the checkpoint.
        Returns the model, the current epoch, and training losses.
        """
        def get_epoch(cp):
            return int(cp.split('epoch=')[-1].split('.checkpoint.pth.tar')[0])

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        cp_files = [file_ for file_ in os.listdir(checkpoint_dir)
                    if file_.startswith('epoch=') and file_.endswith('.checkpoint.pth.tar')]
        cp_files.sort(key=lambda x: get_epoch(x))

        if not cp_files:
            print('No saved model parameters found')
            if force:
                raise Exception('Checkpoint not found')
            else:
                return model, 0, []
        epochs = [get_epoch(cp) for cp in cp_files]

        if not force:
            epochs = [0] + epochs
            print('Which epoch to load from? Choose from epochs below:')
            print(epochs)
            print('Enter 0 to train from scratch.')
            print(">> ", end='')
            inp_epoch = int(input())
            if inp_epoch not in epochs:
                raise Exception("Invalid epoch number")
            if inp_epoch == 0:
                print("Checkpoint not loaded")
                clear_checkpoint(checkpoint_dir)
                return model, 0, []
        else:
            print('Which epoch to load from? Choose from epochs below:')
            print(epochs)
            print(">> ", end='')
            inp_epoch = int(input())
            if inp_epoch not in epochs:
                raise Exception("Invalid epoch number")

        filename = os.path.join(checkpoint_dir, 'epoch={}.checkpoint.pth.tar'.format(inp_epoch))

        print("Loading from checkpoint {}".format(filename))

        if cuda:
            checkpoint = torch.load(filename)
        else:
            checkpoint = torch.load(filename, map_location=lambda storage, loc: storage)

        try:
            stats = checkpoint['stats']
            if pretrain:
                model.load_state_dict(checkpoint['state_dict'], strict=False)
            else:
                model.load_state_dict(checkpoint['state_dict'])
            print("=> Successfully restored checkpoint (trained for {} epochs)".format(checkpoint['epoch']))
        except:
            print("=> Checkpoint not successfully restored")
            raise

        return model, inp_epoch, stats

    def train(config, dataset, model):
        """
        restore the checkpoint of the base model, and evaluate its accuracy on different classes (different duration) of pulses
        """
        # p = 0
        train_loader, val_loader = dataset.train_loader, dataset.val_loader 
        if 'use_weighted' not in config:
            criterion = torch.nn.MSELoss() # Here we diefine the criteron for the evaluate 
            #epoch and evaluate_loop
        else:
            criterion = torch.nn.MSELoss() # how to define weight?
        print('Loading model...') 
        force = config['ckpt_force'] if 'ckpt_force' in config else False
        model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
        y_true, y_pred, running_loss,n,output,y = evaluate_loop(val_loader, model, criterion)

        print('Finished Training')
        return np.sum(running_loss)/len(running_loss),y_true, y_pred, model

    if __name__ == '__main__':
        config = {
            'batch_size': 4,
            'ckpt_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_duration_5_15_2fc_long/transfer',
            'plot_name': 'TransferCPU_4_4e-3_50EP',
            'dict_path': '/content/drive/MyDrive/ResNet18.pt',

        }
        model = load_pretrained(config['dict_path'])
        lin = model.fc
        new_lin = nn.Sequential(
            lin,
            nn.LeakyReLU(0.2),
            nn.Linear(lin.out_features, 1000),
            nn.LeakyReLU(0.2),
            nn.Linear(1000, 128)
        )
        model.fc = new_lin
        model = model.double()
        Loss,y_true, y_pred,model = train(config, dataset, model)  
        model = model.cuda()
        print("The loss of the base model on class %02d is: " % (p+5), Loss)
    ############################
    """
    Freeze all the parameters except for the fc block by setting the requires_grad to False, train the fc block using different classes of pulses
    """
    cntr = 0
    lt = 10
    for child in model.children():
        cntr+=1
        if cntr < lt:
            for param in child.parameters():
                param.requires_grad = False
    #############################


    def save_checkpoint(model, epoch, checkpoint_dir, stats):
        """
        Save model checkpoint.
        """
        state = {
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'stats': stats,
        }

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        filename = os.path.join(checkpoint_dir,'epoch={}.checkpoint.pth.tar'.format(epoch))
        torch.save(state, filename)
        
    def restore_checkpoint(model, checkpoint_dir, cuda=True, force=False, pretrain=True):
        """
        If a checkpoint exists, restores the PyTorch model from the checkpoint.
        Returns the model, the current epoch, and training losses.
        """
        def get_epoch(cp):
            return int(cp.split('epoch=')[-1].split('.checkpoint.pth.tar')[0])

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        cp_files = [file_ for file_ in os.listdir(checkpoint_dir)
                    if file_.startswith('epoch=') and file_.endswith('.checkpoint.pth.tar')]
        cp_files.sort(key=lambda x: get_epoch(x))

        if not cp_files:
            print('No saved model parameters found')
            if force:
                raise Exception('Checkpoint not found')
            else:
                return model, 0, []

        # Find latest epoch
        epochs = [get_epoch(cp) for cp in cp_files]

        if not force:
            epochs = [0] + epochs
            print('Which epoch to load from? Choose from epochs below:')
            print(epochs)
            print('Enter 0 to train from scratch.')
            print(">> ", end='')
            inp_epoch = 0
            #inp_epoch = int(input())
            if inp_epoch not in epochs:
                raise Exception("Invalid epoch number")
            if inp_epoch == 0:
                print("Checkpoint not loaded")
                clear_checkpoint(checkpoint_dir)
                return model, 0, []
        else:
            print('Which epoch to load from? Choose from epochs below:')
            print(epochs)
            print(">> ", end='')
            inp_epoch = int(input())
            if inp_epoch not in epochs:
                raise Exception("Invalid epoch number")

        filename = os.path.join(checkpoint_dir, 'epoch={}.checkpoint.pth.tar'.format(inp_epoch))

        print("Loading from checkpoint {}".format(filename))

        if cuda:
            checkpoint = torch.load(filename)
        else:
            # Load GPU model on CPU
            checkpoint = torch.load(filename, map_location=lambda storage, loc: storage)

        try:
            stats = checkpoint['stats']
            if pretrain:
                model.load_state_dict(checkpoint['state_dict'], strict=False)
            else:
                model.load_state_dict(checkpoint['state_dict'])
            print("=> Successfully restored checkpoint (trained for {} epochs)".format(checkpoint['epoch']))
        except:
            print("=> Checkpoint not successfully restored")
            raise

        return model, inp_epoch, stats

    class Plotter:
        def __init__(self, stats=[], name='CNN'):
            self.stats = stats
            self.name = name
            self.axes = self.make_cnn_training_plot()

        def make_cnn_training_plot(self):
            """
            Runs the setup for an interactive matplotlib graph that logs the loss and accuracy
            """
            print('Setting up interactive graph...')
            plt.ion()
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            plt.suptitle(self.name + ' Training')
            axes[0].set_xlabel('Epoch')
            axes[0].set_ylabel('Accuracy')
            axes[1].set_xlabel('Epoch')
            axes[1].set_ylabel('Loss')
            return axes

        def log_cnn_training(self, epoch):
            """
            Logs the validation accuracy and loss to the terminal
            """
            valid_loss, train_loss = self.stats[-1]
            print('Epoch {}'.format(epoch))
            print('\tValidation Loss: {}'.format(valid_loss))
            print('\tTrain Loss: {}'.format(train_loss))

        def update_cnn_training_plot(self, epoch):
            """
            Updates the training plot with a new data point for loss and accuracy
            """
            xrange = range(epoch - len(self.stats) + 1, epoch + 1)

            self.axes[1].plot(xrange, [s[0] for s in self.stats], linestyle='--', marker='o', color='b')
            self.axes[1].plot(xrange, [s[1] for s in self.stats], linestyle='--', marker='o', color='r')
            self.axes[0].legend(['Validation', 'Train'])
            self.axes[1].legend(['Validation', 'Train'])
            plt.pause(0.00001)

        def save_cnn_training_plot(self):
            """
            Saves the training plot to a file
            """
            plt.savefig(self.name + '_training_plot.png', dpi=200)

        def hold_training_plot(self):
            """
            Keep the program alive to display the training plot
            """
            plt.ioff()
            plt.show()
    class FROGDataset:
        """
        Dog Dataset.
        """
        def __init__(self, batch_size=4):
            self.batch_size = batch_size
            self.train_dataset,self.train_label,self.val_dataset,self.val_label = self.get_train_numpy()
            self.train_loader,self.val_loader = self.get_dataloaders()

        def get_train_numpy(self):
            df = pd.read_pickle('/content/drive/MyDrive/Test_domain_generalization/Pyret_Labels_for_CNN_duration_2000_bin1_%02d.pkl' % (p + 5))

            self.train_dataset = np.tile(np.expand_dims(df.iloc[0,1][:1800,:,:],axis = 1),[1,3,1,1])
            self.train_label = df.iloc[0,0][:1800,:]
            self.val_dataset = np.tile(np.expand_dims(df.iloc[0,1][1800:,:,:],axis = 1),[1,3,1,1])
            self.val_label = df.iloc[0,0][1800:,:]
            return self.train_dataset,self.train_label,self.val_dataset,self.val_label

        def get_dataloaders(self):
            # train set
            self.train_dataset = self.train_dataset.astype(np.double)
            train_label = self.train_label 
            self.train_label  = self.train_label.astype(np.double)
            train_set = torch.tensor(self.train_dataset)
            train_label = torch.tensor(self.train_label)
            dataset = torch.utils.data.TensorDataset(train_set, train_label)
            train_loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=False)
            print(type(train_loader))
            # validation set
            self.val_dataset  = self.val_dataset.astype(np.double)
            self.val_label = self.val_label.astype(np.double)
            val_set = torch.tensor(self.val_dataset)
            val_label = torch.tensor(self.val_label)
            datasetval = torch.utils.data.TensorDataset(val_set, val_label)
            val_loader = torch.utils.data.DataLoader(datasetval, batch_size=self.batch_size, shuffle=False)
            return train_loader,val_loader

    def _train_epoch(train_loader, model, criterion, optimizer):
        for i, (X, y) in enumerate(train_loader):             
            # clear parameter gradients
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad() 
            output = model(X)
            loss = criterion(output, y)#. https://stackoverflow.com/questions/50408673/how-to-build-an-autograd-compatible-pytorch-module-that-resizes-tensors-like-ima
            loss.backward()  
            optimizer.step() 

    def _evaluate_epoch( train_loader, val_loader, model, criterion, epoch,batch_size): 
        """
        Evaluates the model on the train and validation set.
        """
        stat = []
        Worst_Labels = np.zeros((1,128))
        Worst_FROG = np.zeros((1,3,64,64))
        for data_loader in [val_loader, train_loader]:
            if data_loader == val_loader:
                y_true, y_pred, running_loss1,BatchNum_for_evaluate_epoch = evaluate_loop(data_loader, model, criterion) 
                total_loss = np.sum(running_loss1) / BatchNum_for_evaluate_epoch  
                stat += [total_loss]
            else:
                y_true, y_pred, running_loss,BatchNum_for_evaluate_epoch = evaluate_loop(data_loader, model, criterion) 
                Indices_Worst_FROG = np.asarray(running_loss).argsort()[-int(np.floor(600/batch_size)):]
                Loss_Worst = np.sum(np.take(running_loss,np.ndarray.tolist( Indices_Worst_FROG)))
                total_loss = np.sum(running_loss) / BatchNum_for_evaluate_epoch   
                stat += [total_loss]
        return running_loss1, stat

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def evaluate_loop(data_loader, model, criterion):
        model.eval() 
        y_true, y_pred, running_loss = [], [], []
        BatchNum_for_evaluate_epoch=0
        for X, y in data_loader:  
            with torch.no_grad():
                output = model(X)  # output is torch tensor
                y_true.append(y)
                y_pred.append(output)
                BatchNum_for_evaluate_epoch += 1
                if criterion is not None:
                    running = []
                    for i in range(0,output.size()[0]):

                        running.append(criterion(output[i,:], y[i,:]).item() )  # The output [batchsize,num of classes]
                    running_loss += running
        model.train() 
        return y_true, y_pred, running_loss,BatchNum_for_evaluate_epoch

    def load_pretrained(dict_path):
        """
        Load a ResNet-18 model from `torchvision.models` with pre-trained weights. Freeze all the parameters besides the
        final layer by setting the flag `requires_grad` for each parameter to False. Replace the final fully connected layer
        with another fully connected layer with `num_classes` many output units.
        Inputs:
            - num_classes: int
        Returns:
            - model: PyTorch model
        """
        # TODO (part d): load a pre-trained ResNet-18 model
        Resnet18 = models.resnet18(pretrained=False)
        if (dict_path != None):
            Resnet18.load_state_dict(torch.load(dict_path))
        for param in Resnet18.parameters():
            param.requires_grad = True
        n_inputs = Resnet18.fc.in_features
        Resnet18.fc = torch.nn.Linear(512,1000)
        return Resnet18

    def initialize_weights(net):
        for m in net.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.02)
                m.bias.data.zero_()

    def print_network(net):
        num_params = 0
        for param in net.parameters():
            num_params += param.numel()
        print(net)
        print('Total number of parameters: %d' % num_params)

    def imsave1(X,epoch, edge):
        for i in range( 0,edge* edge):
            plt.subplot(edge, edge,  i+1)
            plt.axis('off')
            plt.plot(X[i,0,:])
        plt.subplot(edge, edge,  1)
        filename1 = 'continouse_generated_plot_%02d.png' % (epoch+1)
        plt.savefig(filename1)
        plt.close()

    def train(config, dataset, model):
        train_loader, val_loader = dataset.train_loader, dataset.val_loader 

        if 'use_weighted' not in config:
            criterion = torch.nn.MSELoss() # Here we diefine the criteron for the evaluate 
        else:
            criterion = torch.nn.MSELoss() 
        
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = config['learning_rate'])
        # Attempts to restore the latest checkpoint if exists
        print('Loading model...')
        Train_losses, Statt = [],[]
        for epoch in range(0, config['num_epoch'],2):
            # Train model on training set
            time1 = time.time()
            _train_epoch(train_loader, model, criterion, optimizer)
            time2 = time.time()
            print("time_train_epoch",time2 -time1)
            # Evaluate model on training and validation set
            critetion1 = torch.nn.L1Loss()
            Train_loss, Stat = _evaluate_epoch( train_loader, val_loader, model, criterion1, epoch + 1,config['batch_size'])
            time3 = time.time()
            print("time_evaluate_epoch",time3 -time2)
            Train_losses.append(Train_loss)
            Statt.append(Stat)
            save_checkpoint(model, epoch + 1, config['ckpt_save_path'], Stat)
        return Train_losses,Statt

    if __name__ == '__main__':
        config = {
            'batch_size': 24,
            'ckpt_save_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_%02d_fine_tune_2fc_bin1/transfer' % (p + 5),
            'plot_name': 'Transfer_4e-2_CNN',
            'num_epoch': 80,
            'learning_rate': 4e-5
        }
        dataset = FROGDataset(config['batch_size'])
        Train_losses, Statt = train(config, dataset, model)
        TotalLoss = pd.DataFrame(columns = ['Train_losses','Statt'])
        TotalLoss.loc[0,'Train_losses'] = Train_losses
        TotalLoss.loc[0,'Statt'] = Statt
        if not os.path.exists('/content/drive/MyDrive/FROG training result/Loss/CNN'):
            os.makedirs('/content/drive/MyDrive/FROG training result/Loss/CNN')
        TotalLoss.to_pickle("/content/drive/MyDrive/FROG training result/Loss/CNN/TotalLoss_CNN_4e-4_120_24_%02d_fine_tune_2c_bin1.pkl" % (p + 5))


    #############################
    #### loss plots for all the models. Validate the general improvement
    fine_tune = pd.read_pickle('/content/drive/MyDrive/FROG training result/Loss/CNN/TotalLoss_CNN_4e-4_120_24_%02d_fine_tune_2c_bin1.pkl' % (p+5))
    plt.plot(np.array(fine_tune['Statt'][0])[:,1])
    #############################

    
    def load_pretrained(dict_path):
        """
        Load a ResNet-18 model from `torchvision.models` without pre-trained weights
        """
        Resnet18 = models.resnet18(pretrained=False)
        if (dict_path != None):
            Resnet18.load_state_dict(torch.load(dict_path))
        for param in Resnet18.parameters():
            param.requires_grad = True
        n_inputs = Resnet18.fc.in_features
        Resnet18.fc = torch.nn.Linear(512,1000)
        return Resnet18

    def evaluate_loop(data_loader, model, criterion):
        model.eval() 
        y_true, y_pred, running_loss = [], [], []
        n=0
        for X, y in data_loader: 
            with torch.no_grad():
                output = model(X) 
                y_true.append(y)
                y_pred.append(output)
                n += 1
                if criterion is not None:
                    running_loss.append(criterion(output, y).item() )  # The output [batchsize,num of classes]
        model.train() 
        y_true, y_pred = torch.cat(y_true), torch.cat(y_pred) 
        return y_true, y_pred, running_loss,n,output,y

    def train(config, dataset, model):
        """
        The dataset was already defined by the FROGDataset function earlier
        """
        # Data loaders
        train_loader, val_loader = dataset.train_loader, dataset.val_loader 
        if 'use_weighted' not in config:
            criterion = torch.nn.L1Loss() 
        else:
            criterion = torch.nn.L1Loss() 
        print('Loading model...') 
        force = config['ckpt_force'] if 'ckpt_force' in config else False
        model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
        y_true, y_pred, running_loss,n,output,y = evaluate_loop(val_loader, model, criterion)
        print('Finished Training')
        return np.sum(running_loss)/len(running_loss),y_true, y_pred, model

    if __name__ == '__main__':
        config = {
            'batch_size': 4,
            'ckpt_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_%02d_fine_tune_2fc_bin1/transfer' % (p + 5),
            'plot_name': 'TransferCPU_4_4e-3_50EP',
            'dict_path': '/content/drive/MyDrive/ResNet18.pt',

        }
        model = load_pretrained(config['dict_path'])
        lin = model.fc
        new_lin = nn.Sequential(
            lin,
            nn.LeakyReLU(0.2),
            nn.Linear(lin.out_features, 1000),
            nn.LeakyReLU(0.2),
            nn.Linear(1000, 128)
        )
        model.fc = new_lin
        model = model.double()
        model = model.eval()
        Loss,y_true, y_pred,model = train(config, dataset, model)  
        print("The loss of the model with fine-tuned fc block on class %02d is: " % (p+5), Loss)
        print(np.mean(np.sum(np.square(y_true.cpu().numpy() - y_pred.cpu().numpy()),axis = 1)))


In [ ]:
"""
Concatenate 10 fine-tuned (feature extraction) FC blocks together 
"""

def restore_checkpoint(model, checkpoint_dir, cuda=False, force=False, pretrain=False):
    """
    If a checkpoint exists, restores the PyTorch model from the checkpoint.
    Returns the model, the current epoch, and training losses.
    """
    def get_epoch(cp):
        return int(cp.split('epoch=')[-1].split('.checkpoint.pth.tar')[0])

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    cp_files = [file_ for file_ in os.listdir(checkpoint_dir)
                if file_.startswith('epoch=') and file_.endswith('.checkpoint.pth.tar')]
    cp_files.sort(key=lambda x: get_epoch(x))

    if not cp_files:
        print('No saved model parameters found')
        if force:
            raise Exception('Checkpoint not found')
        else:
            return model, 0, []

    # Find latest epoch
    epochs = [get_epoch(cp) for cp in cp_files]

    if not force:
        epochs = [0] + epochs
        print('Which epoch to load from? Choose from epochs below:')
        print(epochs)
        print('Enter 0 to train from scratch.')
        print(">> ", end='')
        inp_epoch = int(input())
        if inp_epoch not in epochs:
            raise Exception("Invalid epoch number")
        if inp_epoch == 0:
            print("Checkpoint not loaded")
            clear_checkpoint(checkpoint_dir)
            return model, 0, []
    else:
        print('Which epoch to load from? Choose from epochs below:')
        print(epochs)
        print(">> ", end='')
        inp_epoch = int(input())
        if inp_epoch not in epochs:
            raise Exception("Invalid epoch number")

    filename = os.path.join(checkpoint_dir, 'epoch={}.checkpoint.pth.tar'.format(inp_epoch))

    print("Loading from checkpoint {}".format(filename))

    if cuda:
        checkpoint = torch.load(filename)
    else:
        # Load GPU model on CPU
        checkpoint = torch.load(filename, map_location=lambda storage, loc: storage)

    try:
        stats = checkpoint['stats']
        if pretrain:
            model.load_state_dict(checkpoint['state_dict'], strict=False)
        else:
            model.load_state_dict(checkpoint['state_dict'])
        print("=> Successfully restored checkpoint (trained for {} epochs)".format(checkpoint['epoch']))
    except:
        print("=> Checkpoint not successfully restored")
        raise

    return model, inp_epoch, stats
#### Use N_bins to define the number of bins (N_bins = 10 here)
N_bins = 10
fc_layers1_weight = torch.zeros((N_bins,1000,512))
fc_layers2_weight = torch.zeros((N_bins,1000,1000))
fc_layers3_weight = torch.zeros((N_bins,128,1000))
fc_layers1_bias = torch.zeros((N_bins,1000))
fc_layers2_bias = torch.zeros((N_bins,1000))
fc_layers3_bias = torch.zeros((N_bins,128))
for p in range(0,N_bins):

    config = {
            'batch_size': 24,
            'ckpt_path': '/content/drive/MyDrive/checkpointsResNet4e-4_120_24_%02d_fine_tune_2fc_bin1/transfer' % (p + 5),
            'plot_name': 'Transfer_4e-2_CNN',
            'dict_path': '/content/drive/MyDrive/ResNet18.pt',
            'num_epoch': 40,
            'learning_rate': 4e-4,
    }
    model = load_pretrained(config['dict_path'])
    lin = model.fc
    new_lin = nn.Sequential(
        lin,
        nn.LeakyReLU(0.2),
        nn.Linear(lin.out_features, 1000),
        nn.LeakyReLU(0.2),
        nn.Linear(1000, 128),
    )
    model.fc = new_lin
    model = model.double()
    print('Loading model...') 
    force = config['ckpt_force'] if 'ckpt_force' in config else False
    model, start_epoch, stats = restore_checkpoint(model, config['ckpt_path'], force=force)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.cuda()
    fc_layers1_weight[p,:,:] =  model.fc[0].weight
    fc_layers2_weight[p,:,:] =  model.fc[2].weight
    fc_layers3_weight[p,:,:] =  model.fc[4].weight
    fc_layers1_bias[p,:] =  model.fc[0].bias
    fc_layers2_bias[p,:] =  model.fc[2].bias
    fc_layers3_bias[p,:] =  model.fc[4].bias
fc_layers = pd.DataFrame(columns=["fc_layers1_weight","fc_layers2_weight","fc_layers3_weight","fc_layers1_bias","fc_layers2_bias","fc_layers3_bias"])
fc_layers['fc_layers1_weight'] = fc_layers['fc_layers1_weight'].astype(object)
fc_layers.loc[0, 'fc_layers1_weight'] = fc_layers1_weight.cpu().detach()
fc_layers['fc_layers2_weight'] = fc_layers['fc_layers2_weight'].astype(object)
fc_layers.loc[0, 'fc_layers2_weight'] = fc_layers2_weight.cpu().detach()
fc_layers['fc_layers3_weight'] = fc_layers['fc_layers3_weight'].astype(object)
fc_layers.loc[0, 'fc_layers3_weight'] = fc_layers3_weight.cpu().detach()
fc_layers['fc_layers1_bias'] = fc_layers['fc_layers1_bias'].astype(object)
fc_layers.loc[0, 'fc_layers1_bias'] = fc_layers1_bias.cpu().detach()
fc_layers['fc_layers2_bias'] = fc_layers['fc_layers2_bias'].astype(object)
fc_layers.loc[0, 'fc_layers2_bias'] = fc_layers2_bias.cpu().detach()
fc_layers['fc_layers3_bias'] = fc_layers['fc_layers3_bias'].astype(object)
fc_layers.loc[0, 'fc_layers3_bias'] = fc_layers3_bias.cpu().detach()
fc_layers.to_pickle('/content/drive/MyDrive/Test_domain_generalization/fc_layers.pkl')